In [1]:
import pandas as pd
from influxdb import InfluxDBClient

import json
import re
from datetime import datetime
import requests

In [2]:
#optional, jupyter notebook dislay options to see more data
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_colwidth = None
pd.options.display.precision = 15

In [3]:
coin_auth = json.load(open('coin_credentials'))
coin_client = InfluxDBClient(
    host='testresults.qt.io', 
    username=coin_auth['username'],
    password=coin_auth['password'],
    port=8086,
    database='coin')

In [26]:
#q = "select * from flaky_test where branch = 'dev' and time > now() - 2d"
#alternative queries:
q = "select * from flaky_test where branch = 'dev' and time > now() - 5d"
df = pd.DataFrame(coin_client.query(q).get_points())
#since we do not have stats about MacOS we get won't analyse those tests
df.drop(df[df.target_os == 'MacOS'].index, inplace=True)
#some pre-processing to group the tests by : testcase, testfunction and platform on wich it was run:
df['platform'] = df['target_os_version'] + df['target_arch']+ df['target_compiler']
df['full_test_params'] = df['testCase'] + '+' + df['testFunction'] +'+' + df['platform']
df.head()

,time,branch,computeHost,cpu_count,cpu_count_1,cpu_seconds,duration,host_arch,host_compiler,host_os,host_os_version,id,path,platform_id,project,result,sha1,state,target_arch,target_compiler,target_os,target_os_version,taskType,testCase,testFunction,type,platform,full_test_params
28,2023-01-13T13:38:23.552005Z,dev,big-grouse.on1.qt.io,4,None,15532,3883,X86_64,GCC,Linux,openSUSE_15_4,1681435514,qt/qtbase/61849f0f3b67749a5a92f3525bc9c777ffa5085e/LinuxopenSUSE_15_4x86_64LinuxopenSUSE_15_4x86_64GCCqtci-linux-openSUSE-15.4-x86_64-50-11fcc3Sccache_UseConfigure_WarningsAreErrors/6d298dc4711b0382097b2ef17e67ff5d9ca31816/test_1681435522,opensuse-15.4-developer-build,qt/qtbase,Failed,f5f7c3e7ed5ff0804b87202e3ab500b87a037930,Done,X86_64,GCC,Linux,openSUSE_15_4,Integration,tst_qgraphicsproxywidget,clickFocus,Test,openSUSE_15_4X86_64GCC,tst_qgraphicsproxywidget+clickFocus+openSUSE_15_4X86_64GCC
29,2023-01-13T13:38:23.552005Z,dev,big-grouse.on1.qt.io,4,None,15532,3883,X86_64,GCC,Linux,openSUSE_15_4,1681435514,qt/qtbase/61849f0f3b67749a5a92f3525bc9c777ffa5085e/LinuxopenSUSE_15_4x86_64LinuxopenSUSE_15_4x86_64GCCqtci-linux-openSUSE-15.4-x86_64-50-11fcc3Sccache_UseConfigure_WarningsAreErrors/6d298dc4711b0382097b2ef17e67ff5d9ca31816/test_1681435522,opensuse-15.4-developer-build,qt/qtbase,Failed,f5f7c3e7ed5ff0804b87202e3ab500b87a037930,Done,X86_64,GCC,Linux,openSUSE_15_4,Integration,tst_qwidget,hoverPosition,Test,openSUSE_15_4X86_64GCC,tst_qwidget+hoverPosition+openSUSE_15_4X86_64GCC
30,2023-01-13T13:38:23.552005Z,dev,caring-lemur.on1.qt.io,4,None,15824,3956,X86_64,MSVC2019,Windows,Windows_11_22H2,1681435514,qt/qtbase/61849f0f3b67749a5a92f3525bc9c777ffa5085e/WindowsWindows_11_22H2x86_64WindowsWindows_11_22H2x86_64MSVC2019qtci-windows-11_22H2-x86_64-50-8a755cDebugAndRelease_Sccache_UseConfigure_WarningsAreErrors/3f1ea717f4de242a457aff7bf9f20895b7f07b16/test_1681435537,windows-11_22h2-msvc2019-developer-build,qt/qtbase,Failed,f5f7c3e7ed5ff0804b87202e3ab500b87a037930,Done,X86_64,MSVC2019,Windows,Windows_11_22H2,Integration,tst_qwidget,enterLeaveOnWindowShowHide,Test,Windows_11_22H2X86_64MSVC2019,tst_qwidget+enterLeaveOnWindowShowHide+Windows_11_22H2X86_64MSVC2019
31,2023-01-13T13:38:23.552005Z,dev,pinky-pig.on1.qt.io,8,None,21504,2688,X86_64,GCC,Linux,Ubuntu_20_04,1681435514,qt/qtbase/61849f0f3b67749a5a92f3525bc9c777ffa5085e/LinuxUbuntu_20_04x86_64QNXQNX_710x86_64GCCqtci-linux-Ubuntu-20.04-x86_64-50-a897c8UseConfigure/ae7b9b122ebbd265cda03d5393b56cfa685ef2b5/test_1681435561,qnx-710-x86_64-developer-build-on-linux,qt/qtbase,Failed,f5f7c3e7ed5ff0804b87202e3ab500b87a037930,Done,X86_64,QCC,QNX,QNX_710,Integration,tst_qsystemsemaphore,initialValue,Test,QNX_710X86_64QCC,tst_qsystemsemaphore+initialValue+QNX_710X86_64QCC
34,2023-01-13T13:38:23.552005Z,dev,square-tuna.on1.qt.io,4,None,16992,4248,X86_64,Mingw,Windows,Windows_10_22H2,1681435514,qt/qtbase/61849f0f3b67749a5a92f3525bc9c777ffa5085e/WindowsWindows_10_22H2x86_64WindowsWindows_10_22H2x86_64Mingwqtci-windows-10_22H2-x86_64-50-a44c48Sccache_UseConfigure_WarningsAreErrors/fb310d3d5aafc5caf422c12e68dcb3350791d4eb/test_1681435551,windows-10_22h2-mingw11-developer-build,qt/qtbase,Failed,f5f7c3e7ed5ff0804b87202e3ab500b87a037930,Done,X86_64,Mingw,Windows,Windows_10_22H2,Integration,tst_qgraphicsscene,hoverEvents_siblings,Test,Windows_10_22H2X86_64Mingw,tst_qgraphicsscene+hoverEvents_siblings+Windows_10_22H2X86_64Mingw


In [27]:
df['full_test_params']
df2 = df.groupby(['full_test_params'])['full_test_params'].count().to_frame()
df2 = df2.rename(columns={"full_test_params": "count"})
df2.reset_index(inplace=True)
df2.sort_values(by=['count'], inplace = True, ascending = False)
df2.head()


,full_test_params,count
347,tst_qwidget+enterLeaveOnWindowShowHide+Windows_11_22H2X86_64MSVC2019,52
313,tst_qwebenginepage+backgroundColor+Windows_10_22H2X86_64MSVC2019,40
321,tst_qwebengineprofile+clearDataFromCache+Windows_11_22H2X86_64MSVC2019,32
242,tst_qquicktextarea+touchscreenDoesNotSelect+Android_ANYX86Clang,31
66,tst_qgesturerecognizer+panGesture+openSUSE_15_4X86_64GCC,28


In [28]:
df3 = df[df['full_test_params'] == 'tst_qwidget+enterLeaveOnWindowShowHide+Windows_11_22H2X86_64MSVC2019']
df3.shape

(52, 28)

# fidning timestamp when test failed (from log.txt.gz)


In [29]:
path = df3.iloc[0]['path']
url = "https://testresults.qt.io/logs/{}/log.txt.gz".format(path)
#url is path to log.txt.gz with test execution data, where link to telegraf metrics is added
response = requests.get(url)
response

<Response [200]>

In [30]:
df3.head()

,time,branch,computeHost,cpu_count,cpu_count_1,cpu_seconds,duration,host_arch,host_compiler,host_os,host_os_version,id,path,platform_id,project,result,sha1,state,target_arch,target_compiler,target_os,target_os_version,taskType,testCase,testFunction,type,platform,full_test_params
30,2023-01-13T13:38:23.552005Z,dev,caring-lemur.on1.qt.io,4,None,15824,3956,X86_64,MSVC2019,Windows,Windows_11_22H2,1681435514,qt/qtbase/61849f0f3b67749a5a92f3525bc9c777ffa5085e/WindowsWindows_11_22H2x86_64WindowsWindows_11_22H2x86_64MSVC2019qtci-windows-11_22H2-x86_64-50-8a755cDebugAndRelease_Sccache_UseConfigure_WarningsAreErrors/3f1ea717f4de242a457aff7bf9f20895b7f07b16/test_1681435537,windows-11_22h2-msvc2019-developer-build,qt/qtbase,Failed,f5f7c3e7ed5ff0804b87202e3ab500b87a037930,Done,X86_64,MSVC2019,Windows,Windows_11_22H2,Integration,tst_qwidget,enterLeaveOnWindowShowHide,Test,Windows_11_22H2X86_64MSVC2019,tst_qwidget+enterLeaveOnWindowShowHide+Windows_11_22H2X86_64MSVC2019
44,2023-01-13T14:01:16.728448Z,dev,divine-oyster.on1.qt.io,4,None,13688,3422,X86_64,MSVC2019,Windows,Windows_11_22H2,1681433059,qt/qtbase/248917d0726318efd828ecc31c3363c68a27c762/WindowsWindows_11_22H2x86_64WindowsWindows_11_22H2x86_64MSVC2019qtci-windows-11_22H2-x86_64-50-8a755cDebugAndRelease_Sccache_UseConfigure/ed5cab89f998c6de21d8b249b54f3d2636e52b0f/test_1681433443,windows-11_22h2-msvc2019,qt/qtbase,Failed,a478d730f896e89fce83483c8b056b47ae973736,Done,X86_64,MSVC2019,Windows,Windows_11_22H2,Integration,tst_qwidget,enterLeaveOnWindowShowHide,Test,Windows_11_22H2X86_64MSVC2019,tst_qwidget+enterLeaveOnWindowShowHide+Windows_11_22H2X86_64MSVC2019
120,2023-01-13T14:01:16.728448Z,dev,pure-bear.on1.qt.io,4,None,16980,4245,X86_64,MSVC2019,Windows,Windows_11_22H2,1681433059,qt/qtbase/248917d0726318efd828ecc31c3363c68a27c762/WindowsWindows_11_22H2x86_64WindowsWindows_11_22H2x86_64MSVC2019qtci-windows-11_22H2-x86_64-50-8a755cDebugAndRelease_Sccache_UseConfigure_WarningsAreErrors/3f1ea717f4de242a457aff7bf9f20895b7f07b16/test_1681433447,windows-11_22h2-msvc2019-developer-build,qt/qtbase,Failed,a478d730f896e89fce83483c8b056b47ae973736,Done,X86_64,MSVC2019,Windows,Windows_11_22H2,Integration,tst_qwidget,enterLeaveOnWindowShowHide,Test,Windows_11_22H2X86_64MSVC2019,tst_qwidget+enterLeaveOnWindowShowHide+Windows_11_22H2X86_64MSVC2019
280,2023-01-13T15:51:33.725415Z,dev,salivating-bat.on1.qt.io,4,None,12368,3092,X86_64,MSVC2019,Windows,Windows_11_22H2,web_qt_qtbase_1673613638092,qt/qtbase/8c9bf267cc73e0f3d2671d2fc46199d6ecb68e0c/WindowsWindows_11_22H2x86_64WindowsWindows_11_22H2x86_64MSVC2019qtci-windows-11_22H2-x86_64-50-8a755cDebugAndRelease_Sccache_UseConfigure_WarningsAreErrors/3f1ea717f4de242a457aff7bf9f20895b7f07b16/test_1681437808,windows-11_22h2-msvc2019-developer-build,qt/qtbase,Failed,c78be687657526d6d4e37637e6b7fcc87e80d230,Done,X86_64,MSVC2019,Windows,Windows_11_22H2,EarlyWarning,tst_qwidget,enterLeaveOnWindowShowHide,Test,Windows_11_22H2X86_64MSVC2019,tst_qwidget+enterLeaveOnWindowShowHide+Windows_11_22H2X86_64MSVC2019
284,2023-01-13T15:51:33.725415Z,dev,grand-mako.on1.qt.io,4,None,14216,3554,X86_64,MSVC2019,Windows,Windows_11_22H2,web_qt_qtbase_1673613638092,qt/qtbase/8c9bf267cc73e0f3d2671d2fc46199d6ecb68e0c/WindowsWindows_11_22H2x86_64WindowsWindows_11_22H2x86_64MSVC2019qtci-windows-11_22H2-x86_64-50-8a755cDebugAndRelease_Sccache_UseConfigure/ed5cab89f998c6de21d8b249b54f3d2636e52b0f/test_1681437798,windows-11_22h2-msvc2019,qt/qtbase,Failed,c78be687657526d6d4e37637e6b7fcc87e80d230,Done,X86_64,MSVC2019,Windows,Windows_11_22H2,EarlyWarning,tst_qwidget,enterLeaveOnWindowShowHide,Test,Windows_11_22H2X86_64MSVC2019,tst_qwidget+enterLeaveOnWindowShowHide+Windows_11_22H2X86_64MSVC2019


In [31]:
url

'https://testresults.qt.io/logs/qt/qtbase/61849f0f3b67749a5a92f3525bc9c777ffa5085e/WindowsWindows_11_22H2x86_64WindowsWindows_11_22H2x86_64MSVC2019qtci-windows-11_22H2-x86_64-50-8a755cDebugAndRelease_Sccache_UseConfigure_WarningsAreErrors/3f1ea717f4de242a457aff7bf9f20895b7f07b16/test_1681435537/log.txt.gz'

In [32]:
searched_text = "FAIL!  : {}::{}".format(df3.iloc[0]['testCase'],df3.iloc[0]['testFunction'] )
searched_text
#agent:2023/01/13 13:00:36 build.go:403: 522: FAIL!  : tst_QWidget::enterLeaveOnWindowShowHide
#FAIL!  : tst_QWidget::enterLeaveOnWindowShowHide

'FAIL!  : tst_qwidget::enterLeaveOnWindowShowHide'

In [35]:
for line in response.text.split('\n'):
    if searched_text.lower() in line.lower():
        break
line

'agent:2023/01/13 13:00:36 build.go:403: 522: FAIL!  : tst_QWidget::enterLeaveOnWindowShowHide(dialog) Compared values are not the same'

In [40]:
timestamp = line.split(" build.go")[0]
timestamp_str  = timestamp.split("agent:")[1]
timestamp_str

'2023/01/13 13:00:36'

In [55]:
import time
timestamp_str = '2023/01/13 13:00:36'
datetime_timestamp = datetime.strptime(timestamp_str, '%Y/%m/%d %H:%M:%S')
datetime_timestamp, time.mktime(datetime_timestamp.timetuple())

(datetime.datetime(2023, 1, 13, 13, 0, 36), 1673611236.0)

# connecting  given test data with vm data 

In [5]:
#just taking first random test:
path = df.iloc[0]['path']
url = "https://testresults.qt.io/logs/{}/log.txt.gz".format(path)
#url is path to log.txt.gz with test execution data, where link to telegraf metrics is added
response = requests.get(url)
response

<Response [200]>

In [6]:
#ugly processing of respone to find the start timestamp. end timestamp. and server id
#I search for:
#INFO:workitem.py:760(23870): <a href="https://inframetrics.intra.qt.io/d/nOAsINmZz/telegraf-coin-vms?orgId=1&from=1673829274000&to=1673830968000&var-server=1681269718-122700">VM metrics
result = re.findall(r"INFO:workitem.py:[0-9]+\([0-9]+\):.+VM metrics", response.text)
result
start_time = re.findall(r"from=[0-9]+", result[0])
start_timestamp = start_time[0].split('from=')[1]
end_time = re.findall(r"to=[0-9]+", result[0])
end_timestamp = end_time[0].split('to=')[1]
server_id = re.findall(r"server=[0-9]+-[0-9]+", result[0])
server_id = server_id[0].split('server=')[1]
#start_timestamp is in Unix seconds, you can convert it to human readable time
start_timestamp, end_timestamp, server_id

('1673862903000', '1673863535000', '1681269718-129030')

In [7]:
#connecting to telegraf
telegraf_auth = json.load(open('telegraf_credentials'))
telegraf_client = InfluxDBClient(
    host="qtinfluxdb01.intra.qt.io",
    username=telegraf_auth['username'],
    password=telegraf_auth['password'],
    port=8086,
    ssl=True,
    database="coin_vms")

In [8]:
#all vm mesurements we keep
q = "show measurements"
tdf = pd.DataFrame(telegraf_client.query(q).get_points())
tdf

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


,name
0,cpu
1,disk
2,diskio
3,ioping
4,kernel
5,kernel_vmstat
6,mem
7,netstat
8,processes
9,procstat_lookup


In [9]:
#example query
q = "SELECT *  FROM mem WHERE host =~ /{}(-|$)/ AND time >= {}ms and time <= {}ms".format(server_id,start_timestamp, end_timestamp)
tdf = pd.DataFrame(telegraf_client.query(q).get_points())
tdf.head()

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


,time,active,available,available_percent,buffered,cached,commit_limit,committed_as,dirty,free,high_free,high_total,host,huge_page_size,huge_pages_free,huge_pages_total,inactive,low_free,low_total,mapped,page_tables,shared,slab,swap_cached,swap_free,swap_total,total,used,used_percent,vmalloc_chunk,vmalloc_total,vmalloc_used,wired,write_back,write_back_tmp
0,2023-01-16T09:55:20Z,0,7898918912,81.742812284351572,0,0,0,0,0,0,0,0,1681269718-129030-Windows_10_22H2,0,0,0,0,0,0,0,0,0,0,0,0,0,9663135744,1764216832,18.257187715648424,0,0,0,0,0,0
1,2023-01-16T09:55:30Z,0,7518089216,77.801755198027777,0,0,0,0,0,0,0,0,1681269718-129030-Windows_10_22H2,0,0,0,0,0,0,0,0,0,0,0,0,0,9663135744,2145046528,22.198244801972223,0,0,0,0,0,0
2,2023-01-16T09:55:40Z,0,7617515520,78.830679003240135,0,0,0,0,0,0,0,0,1681269718-129030-Windows_10_22H2,0,0,0,0,0,0,0,0,0,0,0,0,0,9663135744,2045620224,21.169320996759868,0,0,0,0,0,0
3,2023-01-16T09:55:50Z,0,7438950400,76.982778645316728,0,0,0,0,0,0,0,0,1681269718-129030-Windows_10_22H2,0,0,0,0,0,0,0,0,0,0,0,0,0,9663135744,2224185344,23.017221354683269,0,0,0,0,0,0
4,2023-01-16T09:56:00Z,0,7381303296,76.386211386745472,0,0,0,0,0,0,0,0,1681269718-129030-Windows_10_22H2,0,0,0,0,0,0,0,0,0,0,0,0,0,9663135744,2281832448,23.613788613254524,0,0,0,0,0,0


In [10]:
#how many datapoints there are for give time slice (start_timestamp, end_timestamp))
tdf.shape

(62, 35)

In [11]:
q = "SELECT *  FROM cpu WHERE host =~ /{}(-|$)/ AND time >= {}ms and time <= {}ms".format(server_id,start_timestamp, end_timestamp)

tdf = pd.DataFrame(telegraf_client.query(q).get_points())
tdf.head()

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


,time,cpu,host,usage_guest,usage_guest_nice,usage_idle,usage_iowait,usage_irq,usage_nice,usage_softirq,usage_steal,usage_system,usage_user
0,2023-01-16T09:55:30Z,cpu-total,1681269718-129030-Windows_10_22H2,0.0,0.0,33.520900321543408,0.0,0.000000000000000,0.0,0.0,0.0,21.704180064308680,44.774919614147912
1,2023-01-16T09:55:30Z,cpu0,1681269718-129030-Windows_10_22H2,0.0,0.0,31.093750000000000,0.0,0.000000000000000,0.0,0.0,0.0,27.187500000000000,41.718750000000000
2,2023-01-16T09:55:30Z,cpu1,1681269718-129030-Windows_10_22H2,0.0,0.0,33.283582089552240,0.0,4.477611940298507,0.0,0.0,0.0,16.567164179104477,45.671641791044777
3,2023-01-16T09:55:40Z,cpu-total,1681269718-129030-Windows_10_22H2,0.0,0.0,45.631825273010918,0.0,0.000000000000000,0.0,0.0,0.0,35.257410296411855,19.110764430577223
4,2023-01-16T09:55:40Z,cpu0,1681269718-129030-Windows_10_22H2,0.0,0.0,45.638629283489095,0.0,0.155763239875389,0.0,0.0,0.0,38.317757009345797,15.887850467289720


In [12]:
#Tony Sarajavi queries copied from his graana
q = "SELECT mean(total) as total, mean(buffered) as buffered, mean(cached) as cached, mean(total)-mean(free) as non_free, mean(used) as used, mean(total)-mean(available) as non_available  FROM mem WHERE host =~ /1681269718-122700(-|$)/ AND time >= 1673829274000ms and time <= 1673830968000ms GROUP BY time(30s), host ORDER BY asc"
tdf = pd.DataFrame(telegraf_client.query(q).get_points())
tdf.head()

/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qtinfluxdb01.intra.qt.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


,time,total,buffered,cached,non_free,used,non_available
0,2023-01-16T00:34:30Z,16105586688.0,0.0,0.0,16105586688.0,1.795305472000000e+09,1.795305472000000e+09
1,2023-01-16T00:35:00Z,16105586688.0,0.0,0.0,16105586688.0,2.040752810666667e+09,2.040752810666666e+09
2,2023-01-16T00:35:30Z,16105586688.0,0.0,0.0,16105586688.0,2.493913770666667e+09,2.493913770666666e+09
3,2023-01-16T00:36:00Z,16105586688.0,0.0,0.0,16105586688.0,2.725780138666667e+09,2.725780138666666e+09
4,2023-01-16T00:36:30Z,16105586688.0,0.0,0.0,16105586688.0,2.993198421333333e+09,2.993198421333334e+09
